In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
%run "../../config/config"


In [0]:

# import or use variable/function from other notebook and python  file
# Note: - magic commands hsould be at the top of the file and shoud be only content in that cell
raw_folder_location

In [0]:
# %run cannot be used to import a .py file into a notebook. 
# For Python files, use import (after making sure the file is in the workspace or sys.path)
# from utils.utils import *
import sys , os
# sys.path.append("../../utils")
print(os.getcwd())
from utils.utils import *

print(callable(add_ingestion_date)) 

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

schema = StructType([
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("stop", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("milliseconds", IntegerType(), True)
])
# need to do multiline true otherwise dat displayes is null for multiline json
df = spark.read.schema(schema).option("multiline", "true").json("abfss://raw@databdls.dfs.core.windows.net/pit_stops.json")
display(df.show(5, truncate=False))  # Shows full content of each column

In [0]:
# dbutils.widgets.remove("p_data_source")
dbutils.widgets.text("p_data_source", "default_ingestion_date", "Label for date")
v_data_source = dbutils.widgets.get("p_data_source")
print("Input parameter value : -", v_data_source)

In [0]:
from pyspark.sql.functions import current_date

# Add ingestion date column
df_with_ingestion = add_ingestion_date(df,v_data_source)
# Add ingestion date column
# Save to Parquet file
df_with_ingestion.write.mode("overwrite").parquet("abfss://processed@databdls.dfs.core.windows.net/pit_stops_parquet")

# Read from Parquet file
df_parquet = spark.read.parquet("abfss://processed@databdls.dfs.core.windows.net/pit_stops_parquet")
display(df_parquet.show(5, truncate=False))

In [0]:
dbutils.notebook.help()

In [0]:
dbutils.notebook.exit(
    f"Success - {dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()}"
)
